In [30]:
from sklearn import datasets
import pandas as pd
import numpy as np
# import some data to play with
iris = datasets.load_iris()

In [31]:
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [32]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [33]:
# df[np.random.randint():'sepal length (cm)']
df.loc[df.sample(10, random_state=32).index,'sepal length (cm)'] = np.nan

In [34]:
df[~pd.isnull(df).any(axis=1)]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [35]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

imp_mean.fit(df[~pd.isnull(df).any(axis=1)])
X = df[pd.isnull(df).any(axis=1)]
res = imp_mean.transform(X)

In [42]:
df[pd.isnull(df).any(axis=1)].index

array([ 21,  22,  23,  26,  34,  55,  56,  85, 131, 134], dtype=int64)